In [3]:
import os

!pip install nibabel
!pip  install --extra-index-url https://download.pytorch.org/whl/cu117 torch==2.0.0 torchvision==0.15.0 omegaconf torchmetrics==0.10.3 fvcore iopath 
#!pip  install --extra-index-url https://download.pytorch.org/whl/cu117 torch==2.0.0 torchvision==0.15.0 omegaconf torchmetrics==0.10.3 fvcore iopath xformers==0.0.18 submitit
!pip install "numpy<2.0"
#xformers should not be installed!!!!!!!
#!pip uninstall xformers -y

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
  Using cached https://download.pytorch.org/whl/cu117/torch-2.0.0%2Bcu117-cp310-cp310-linux_x86_64.whl (1843.9 MB)
  Using cached https://download.pytorch.org/whl/cu117/torchvision-0.15.0%2Bcu117-cp310-cp310-linux_x86_64.whl (6.1 MB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached torchmetrics-0.10.3-py3-none-any.whl.metadata (15 kB)
  Using cached fvcore-0.1.5.post20221221-py3-none-any.whl
  Using cached iopath-0.1.10-py3-none-any.whl
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Using cached cmake-4.0.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.3 kB)
  Using cached lit-18.1.8-py3-none-any.whl.metadata (2.5 kB)
  Using cached antlr4

In [6]:
import subprocess
import pandas as pd

# Load and prepare the balanced validation set
df = pd.read_csv("/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_test_.csv")

# Print original validation set size
print("Original validation set size:", df[df["split"]=="val"].shape)

val_df = df[df.split == 'val'].copy()
val_df = val_df[val_df.file_path.notna()].reset_index(drop=True)

# Get positive and negative samples for 6-year-cancer
pos_samples = val_df[val_df['6-year-cancer'] == 1]
neg_samples = val_df[val_df['6-year-cancer'] == 0]

print("\nBefore balancing:")
print(f"Positive samples: {len(pos_samples)}")
print(f"Negative samples: {len(neg_samples)}")

# Calculate sizes for balanced set
n_pos = len(pos_samples)
n_neg = len(neg_samples)
target_size = min(n_pos, n_neg)

# Sample equally from positive and negative
balanced_pos = pos_samples.sample(n=target_size, random_state=42)
balanced_neg = neg_samples.sample(n=target_size, random_state=42)

# Combine balanced samples
balanced_val_df = pd.concat([balanced_pos, balanced_neg]).reset_index(drop=True)

print("\nAfter balancing:")
print(f"Balanced validation set shape: {balanced_val_df.shape}")

# Create new dataframe correctly
df_new = df.copy()
# Remove all validation samples
df_new = df_new[df_new.split != 'val']
# Add balanced validation samples
balanced_val_df['split'] = 'val'  # Ensure split column is set
df_new = pd.concat([df_new, balanced_val_df], ignore_index=True)

print("\nFinal shapes:")
print(f"New validation set shape: {df_new[df_new['split']=='val'].shape}")
print(f"Total dataset shape: {df_new.shape}")

# Save the balanced validation set
temp_csv_path = "/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_test_balanced.csv"
df_new.to_csv(temp_csv_path, index=False)

# Run the training script with the balanced validation set
subprocess.run([
    "python3", 
    "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/finetuneA100_gradual_transformer.py",
    "--csv", temp_csv_path,
    "--accum-steps", "10",
    "--cuda-id", "0",
], check=True)

Original validation set size: (5460, 21)

Before balancing:
Positive samples: 272
Negative samples: 1816

After balancing:
Balanced validation set shape: (544, 21)

Final shapes:
New validation set shape: (544, 21)
Total dataset shape: (38936, 21)


/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/rsrch1/ip/msalehjahromi/codes/dinov2-torchrun-dataloader6/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


dict_keys(['teacher'])
Using CUDA device: 0
Class weights: tensor([33.4192, 19.4125, 14.0958, 11.3240,  7.4886,  4.0204])
Using device: cuda:0

>>> Epoch 1/10
Epoch 1: Base model is fully frozen


Traceback (most recent call last):
  File "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/finetuneA100_gradual_transformer.py", line 603, in <module>
    main(args) 
  File "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/finetuneA100_gradual_transformer.py", line 565, in main
    trainer.fit(train_loader, val_loader, 
  File "/rsrch1/ip/msalehjahromi/codes/FineTune/multiGPU/finetuneA100_gradual_transformer.py", line 362, in fit
    for step, (chunks, labels, mask) in enumerate(train_loader, 1):
  File "/opt/venv/jupyter/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 634, in __next__
    data = self._next_data()
  File "/opt/venv/jupyter/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1329, in _next_data
    idx, data = self._get_data()
  File "/opt/venv/jupyter/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1295, in _get_data
    success, data = self._try_get_data()
  File "/opt/venv/jupyter/lib/python3.10/site-packages/tor

KeyboardInterrupt: 